# Preprocesamiento y Análisis de Datos

En este cuaderno, encontrarás todo el código necesario para llevar a cabo un análisis exploratorio de datos con el propósito de caracterizarlos. También, evaluaremos el equilibrio del conjunto de datos y determinaremos el método más adecuado para preparar los datos y construir un modelo con la máxima eficiencia.

In [2]:
# Importar librerpias necesarias para ejecución del código
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.legend import Collection

#Clasificación
import sklearn
from sklearn.decomposition import PCA #Libreria para aplicar PCA

import seaborn as sns
from sklearn.preprocessing import LabelEncoder



/tmp/ipykernel_42458/4182924232.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Dataset original


In [3]:
#Lectura de el archivo
archivo = 'train_data.csv'
data=pd.read_csv(archivo,sep=';')
data

,id_colaborador,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,fecha_nacimiento,salario,performance_score,psi_score,fecha_incorporacion,estado_civil,abandono_6meses
0,100247,102074.0,1,Híbrida,1.760,1,Mujer,Ferias & Networking,1,25/11/1999,140011,99.0,75,25/01/2018,Soltero,0
1,103355,102115.0,1,Híbrida,0.760,2,Hombre,Ferias & Networking,2,16/10/1996,182774,99.0,81,24/07/2021,Soltero,1
2,100669,102060.0,1,Híbrida,4.950,3,Mujer,Referidos,11,29/01/1971,682106,96.0,74,13/07/2016,Viudo,0
3,103760,102062.0,1,Híbrida,13.030,2,Hombre,Linkedin,2,28/04/1992,270232,96.0,82,21/07/2014,Soltero,1
4,100965,102062.0,1,Híbrida,13.045,2,Hombre,Linkedin,2,3/05/1992,266804,95.0,82,22/07/2014,Soltero,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,103567,102171.0,1,Presencial,1.965,8,Mujer,Portal Web,3,20/12/1976,281159,NaN,65,22/12/2023,Casado,1
2148,104098,102172.0,1,Presencial,0.805,12,Mujer,Linkedin,13,23/10/1976,460337,NaN,82,26/07/2023,Casado,1
2149,103987,102155.0,1,Presencial,1.625,1,Hombre,Portal Web,11,19/03/1972,633879,NaN,66,24/07/2023,Soltero,1
2150,103810,102141.0,1,Presencial,5.665,2,Hombre,Portal Web,6,7/04/1971,793977,NaN,80,31/08/2023,Casado,1


In [4]:
# Conteo de cada clase para descripción del dataset
print("Contedo de muestras de cada clase")
print(data['abandono_6meses'].value_counts())
#Medidas estadisticas:
print("Promedio:",np.mean(np.array(data['abandono_6meses'].value_counts())))
print("Desviación",np.std(data['abandono_6meses'].value_counts()))

Contedo de muestras de cada clase
abandono_6meses
0    1149
1    1003
Name: count, dtype: int64
Promedio: 1076.0
Desviación 73.0


## Creación de nuevo dataset

In [5]:

label_encoder = LabelEncoder()

# Cambio a texto
data["modalidad_trabajo"] = label_encoder.fit_transform(data["modalidad_trabajo"])
data["canal_reclutamiento"] = label_encoder.fit_transform(data["canal_reclutamiento"])
data["fecha_nacimiento"] = label_encoder.fit_transform(data["fecha_nacimiento"])
data["fecha_incorporacion"] = label_encoder.fit_transform(data["fecha_incorporacion"])
data["genero"] = label_encoder.fit_transform(data["genero"])
data["estado_civil"] = label_encoder.fit_transform(data["estado_civil"])

#Guardar el nuevo conjunto de datos
data.to_csv('train_data_encoding.csv', index=False)
data

,id_colaborador,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,fecha_nacimiento,salario,performance_score,psi_score,fecha_incorporacion,estado_civil,abandono_6meses
0,100247,102074.0,1,0,1.760,1,1,0,1,1184,140011,99.0,75,905,2,0
1,103355,102115.0,1,0,0.760,2,0,0,2,514,182774,99.0,81,876,2,1
2,100669,102060.0,1,0,4.950,3,1,4,11,1372,682106,96.0,74,242,3,0
3,103760,102062.0,1,0,13.030,2,0,2,2,1330,270232,96.0,82,721,2,1
4,100965,102062.0,1,0,13.045,2,0,2,2,1451,266804,95.0,82,778,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,103567,102171.0,1,1,1.965,8,1,3,3,878,281159,NaN,65,802,0,1
2148,104098,102172.0,1,1,0.805,12,1,2,13,1046,460337,NaN,82,989,0,1
2149,103987,102155.0,1,1,1.625,1,0,3,11,693,633879,NaN,66,878,2,1
2150,103810,102141.0,1,1,5.665,2,0,3,6,1804,793977,NaN,80,1283,0,1


In [4]:
import pandas as pd
import numpy as np

# Asumo que tu dataset se llama 'data' y es un archivo CSV, cambia el nombre y formato según sea necesario
data = pd.read_csv('train_data_encoding.csv')

# Obtener información sobre los datos faltantes
print("Información inicial:")
print(data.isnull().sum())

# Eliminar las filas con datos faltantes
data = data.dropna()

# Mostrar información sobre los datos faltantes después de la limpieza
print("\nInformación después de la limpieza:")
print(data.isnull().sum())

# Guardar el dataset limpio en un nuevo archivo CSV
data.to_csv('data_clean.csv', index=False)


Información inicial:
id_colaborador           0
id_ultimo_jefe          91
seniority                0
modalidad_trabajo        0
distancia_oficina        0
dias_baja_salud          0
genero                   0
canal_reclutamiento      0
permanencia_promedio     0
fecha_nacimiento         0
salario                  0
performance_score       68
psi_score                0
fecha_incorporacion      0
estado_civil             0
abandono_6meses          0
dtype: int64

Información después de la limpieza:
id_colaborador          0
id_ultimo_jefe          0
seniority               0
modalidad_trabajo       0
distancia_oficina       0
dias_baja_salud         0
genero                  0
canal_reclutamiento     0
permanencia_promedio    0
fecha_nacimiento        0
salario                 0
performance_score       0
psi_score               0
fecha_incorporacion     0
estado_civil            0
abandono_6meses         0
dtype: int64


In [5]:
# Conteo de cada clase para descripción del dataset
print("Contedo de muestras de cada clase")
print(data['abandono_6meses'].value_counts())
#Medidas estadisticas:
print("Promedio:",np.mean(np.array(data['abandono_6meses'].value_counts())))
print("Desviación",np.std(data['abandono_6meses'].value_counts()))

Contedo de muestras de cada clase
abandono_6meses
0    1113
1     882
Name: count, dtype: int64
Promedio: 997.5
Desviación 115.5


# Análisis de variables

In [6]:
data

,id_colaborador,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,fecha_nacimiento,salario,performance_score,psi_score,fecha_incorporacion,estado_civil,abandono_6meses
0,100247,102074.0,1,0,1.760,1,1,0,1,1184,140011,99.0,75,905,2,0
1,103355,102115.0,1,0,0.760,2,0,0,2,514,182774,99.0,81,876,2,1
2,100669,102060.0,1,0,4.950,3,1,4,11,1372,682106,96.0,74,242,3,0
3,103760,102062.0,1,0,13.030,2,0,2,2,1330,270232,96.0,82,721,2,1
4,100965,102062.0,1,0,13.045,2,0,2,2,1451,266804,95.0,82,778,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,100824,102147.0,1,1,6.510,8,1,3,15,744,294641,12.0,79,383,2,1
2096,103863,102147.0,1,1,6.520,8,1,3,15,413,301334,12.0,79,491,2,1
2097,102945,102154.0,1,1,1.210,1,0,3,6,1457,738060,12.0,77,613,1,0
2098,103973,102157.0,1,1,4.735,2,1,2,12,987,715749,11.0,81,826,2,1


In [7]:

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


# Separar las características y la etiqueta
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)

# Entrenar el modelo
svm_model.fit(X_train, y_train)

# Predecir las etiquetas para el conjunto de prueba
y_pred = svm_model.predict(X_test)

# Mostrar la matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

# Mostrar el informe de clasificación
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))
